In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df_m = pd.read_csv("/Users/seangao/Desktop/2019 Spring/MGMT 782 Business Research Methods II/Project/tmdb_5000_movies.csv")

# PART 1: Movie Recommendation

# JSON objects are surrounded by curly braces {}. 
# JSON objects are written in key/value pairs.

# Convert json to string for column 'genres' and 'keywords'
df_m['genres'] = df_m['genres'].apply(json.loads)
for index,i in zip(df_m.index,df_m['genres']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    df_m.loc[index,'genres']=str(list1)

df_m['keywords'] = df_m['keywords'].apply(json.loads)
for index,i in zip(df_m.index,df_m['keywords']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    df_m.loc[index,'keywords']=str(list1)   

# Remove unwanted part in the column 'genres' and 'keywords'
df_m['genres'] = df_m['genres'].str.strip('[]').str.replace("'",'').str.replace('u','').str.replace(' ','')
df_m['keywords'] = df_m['keywords'].str.strip('[]').str.replace("u'",'').str.replace("'",'').str.replace(' ','')

# Do the same for 'production_companies', 'production_countries', and 'spoken_languages'
df_m['production_companies'] = df_m['production_companies'].apply(json.loads)
for index,i in zip(df_m.index,df_m['production_companies']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    df_m.loc[index,'production_companies']=str(list1)

df_m['production_companies'] = df_m['production_companies'].str.strip('[]').str.replace("u'",'').str.replace("'",'').str.replace(' ','')
    
df_m['production_countries'] = df_m['production_countries'].apply(json.loads)
for index,i in zip(df_m.index,df_m['production_countries']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    df_m.loc[index,'production_countries']=str(list1)

df_m['production_countries'] = df_m['production_countries'].str.strip('[]').str.replace("u'",'').str.replace("'",'').str.replace(' ','') 
    
df_m['spoken_languages'] = df_m['spoken_languages'].apply(json.loads)
for index,i in zip(df_m.index,df_m['spoken_languages']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    df_m.loc[index,'spoken_languages']=str(list1)

df_m['spoken_languages'] = df_m['spoken_languages'].str.strip('[]').str.replace("u'",'').str.replace("'",'').str.replace(' ','')

In [3]:
# Do the same for column 'cast' in dataset df_c
df_c = pd.read_csv("/Users/seangao/Desktop/2019 Spring/MGMT 782 Business Research Methods II/Project/tmdb_5000_credits.csv")

df_c['cast'] = df_c['cast'].apply(json.loads)
for index,i in zip(df_c.index,df_c['cast']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    df_c.loc[index,'cast']=str(list1)

df_c['cast'] = df_c['cast'].str.strip('[]').str.replace("u'",'').str.replace('u"','').str.replace('"','').str.replace("'",'').str.replace(' ','')

# Extract key crew members from column 'crew'
df_c['crew'] = df_c['crew'].apply(json.loads)
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
df_c['director'] = df_c['crew'].apply(director)

df_c['director'] = df_c['director'].str.replace(' ','')

def composer(x):
    for i in x:
        if i['job'] == 'Original Music Composer':
            return i['name']
df_c['original_music_composer'] = df_c['crew'].apply(composer)

df_c['original_music_composer'] = df_c['original_music_composer'].str.replace(' ','')

def screenplay(x):
    for i in x:
        if i['job'] == 'Screenplay':
            return i['name']
df_c['screenplay'] = df_c['crew'].apply(screenplay)

df_c['screenplay'] = df_c['screenplay'].str.replace(' ','')

def editor(x):
    for i in x:
        if i['job'] == 'Editor':
            return i['name']
df_c['editor'] = df_c['crew'].apply(editor)

df_c['editor'] = df_c['editor'].str.replace(' ','')

df_c.rename(columns={'movie_id':'id'},inplace=True)

df = pd.merge(df_m[['id','budget','genres','keywords',
                    'original_language','popularity','production_companies',
                    'production_countries','revenue','runtime','spoken_languages',
                    'title','vote_average','vote_count']],
              df_c[['id','cast','director','original_music_composer','screenplay',
                    'editor']],how='left')

In [4]:
df.fillna(value = 'NaN', inplace = True)

In [5]:
df['keycrews'] = df["director"] + ',' + df["original_music_composer"] + ',' + df["screenplay"] + ',' + df["editor"]

df['allfeatures'] = df['keywords'] + ',' + df['genres'] + ',' +  df['production_companies']  + ',' + df['production_countries']  + ',' + df['spoken_languages']  + ',' + df['cast']  + ',' + df['keycrews']

count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(df['allfeatures'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(df.index, index = df['title']).drop_duplicates()

def get_recommendations(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

In [9]:
get_recommendations('Iron Man', cosine_sim)

79                              Iron Man 2
31                              Iron Man 3
129                                   Thor
7                  Avengers: Age of Ultron
182                                Ant-Man
169     Captain America: The First Avenger
16                            The Avengers
511                                  X-Men
85     Captain America: The Winter Soldier
126                   Thor: The Dark World
Name: title, dtype: object